In [1]:
import numpy as np
import pandas as pd
import time
import glob
import os
from IPython.display import clear_output
from numpy.fft import rfft, irfft, rfftfreq
import plasma

In [2]:
indir = '/tigress/wvdp/processed_shots/'
outdir = '/tigress/wvdp/processed_modified_shots/'

signal_group = "signal_group_3236450206765786377241194018831785328"
signals_to_modify = ["Locked mode amplitude"]
# Fraction of noise added to the original signal, 0 means keep the original and is useless
noise_amp = [-1, -0.5, 0.5, 1.0, 2.0]
noise_amp_labs = ['denoised', 'reduced', 'addedhalf', 'double', 'triple']

# Make the folders for this signal group and the signals we are modifying
directory = outdir+signal_group
if not os.path.exists(directory):
    os.makedirs(directory)
    
for sig in signals_to_modify:  
    directory += '/'+sig.replace(" ", "")
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    for noise_amp_lab in noise_amp_labs:
        directory_amp = directory + "/"+str(noise_amp_lab)
        if not os.path.exists(directory_amp):
            os.makedirs(directory_amp)

In [3]:
fft_params = {'pad_width': 500,
              'pad_type': 'symmetric',
              'pad_reflect': 'odd',
              'filter_max_hz': 5,
              'time_step': 1e-3,
              'noise_clip_percentiles': [5,95]}

In [4]:
shots_dir = indir+signal_group
all_files = glob.glob(shots_dir+'/*.npz')
num_total = len(all_files)
print(num_total)

5524


In [5]:
def calculate_noise(sig, params):
    sig_odd = len(sig)%2==1
    if sig_odd:
        sig = np.insert(sig,0,sig[0])
    padded_sig = np.pad(sig, 
                        params['pad_width'], 
                        params['pad_type'], 
                        reflect_type=params['pad_reflect'])
    
    # Determine the filter size:
    freq_padded = rfftfreq(len(padded_sig), d=params['time_step'])
    filter_size = np.argmax(freq_padded>params['filter_max_hz'])
    print(filter_size)
    fftsig = rfft(padded_sig)
    fftsig_noise = np.zeros(len(fftsig), dtype='complex64')
    fftsig_noise[filter_size:] = fftsig[filter_size:]
    ifftsig_noise = irfft(fftsig_noise)
    noise = ifftsig_noise[params['pad_width']:-params['pad_width']]
    lower, upper = np.percentile(noise,params['noise_clip_percentiles'])
    noise_clip = np.clip(noise,lower,upper)
    if sig_odd:
        noise_clip = noise_clip[1:]
    return noise_clip

def write_npz(original_file, sig_object, sig_mod, shot_nr, amp_lab):
    file_name = outdir+signal_group+'/'+sig_object.description.replace(" ", "")+'/'+amp_lab+'/'+shot_nr
    if not os.path.exists(file_name):
        old_sig = original_file["signals_dict"].item(0)[sig_object].copy()
        original_file["signals_dict"].item(0)[sig_object]=sig_mod
        new_sig = original_file["signals_dict"].item(0)[sig_object].copy()
        np.savez(file_name, **original_file)
    else:
        print('File already existed ', sig_object.description, shot_nr, amp_lab)

def find_sig_index(fname, signals_to_modify):
    file = np.load(fname, allow_pickle=True)
    signals_dict = list(file.f.signals_dict.item(0).items())
    sig_names = [n[0].description for n in signals_dict]
    sig_indexes = [sig_names.index(s) for s in signals_to_modify]
    sig_objects = [ list(file["signals_dict"].item(0).keys())[i] for i in sig_indexes]
    return sig_indexes, sig_objects

In [6]:
signals_index_to_modify, signals_objects_to_modify = find_sig_index(all_files[0], signals_to_modify)
print(signals_index_to_modify, signals_objects_to_modify)

all signals (determines which signals are downloaded and preprocessed):
dict_values([q95 safety factor, internal inductance, plasma current, Normalized Beta, stored energy, Locked mode amplitude, Plasma density, Radiated Power Core, Radiated Power Edge, Radiated Power, Input Power (beam for d3d), Input Beam Torque, stored energy time derivative, plasma current direction, plasma current target, plasma current error, Electron temperature profile, Electron density profile])
[4] [Locked mode amplitude]


In [7]:
start = time.time()

for i,shot in enumerate(all_files):
    file = dict(np.load(shot, allow_pickle=True))
    if not file['valid']:
        continue
    signals_dict = file["signals_dict"].item(0)

    for sig_object in signals_objects_to_modify:
        sig = signals_dict[sig_object].flatten()
        noise = calculate_noise(sig, fft_params)
        for amp, amp_lab in zip(noise_amp, noise_amp_labs):
            sig_mod = sig+amp*noise
            sig_mod = np.reshape(sig_mod, signals_dict[sig_object].shape)
            shot_nr = shot.split('/')[-1]
            write_npz(file, sig_object, sig_mod, shot_nr, amp_lab)
            
    clear_output(wait=True)
    print('Shots:\t',i, '/',num_total)
    
end = time.time()
print('Time passed: {:.1f}s'.format(end - start))

Shots:	 5523 / 5524
Time passed: 459.5s
